##### Exercise 18-5
Write a method called `deal!` that takes three parameters, a `Deck`, the number of hands and the number of cards per hand. It should create the appropriate number of `Hand` objects, deal the appropriate number of cards per hand, and return an array of `Hand`s.

##### Exercise 18-6

The following are the possible hands in poker, in increasing order of value and decreasing order of probability:

__pair__
    two cards with the same rank

__two pair__
    two pairs of cards with the same rank

__three of a kind__
    three cards with the same rank

__straight__
    five cards with ranks in sequence (aces can be high or low, so Ace-2-3-4-5 is a straight and so is 10-Jack-Queen-King-Ace, but Queen-King-Ace-2-3 is not.)

__flush__
    five cards with the same suit

__full house__
    three cards with one rank, two cards with another

__four of a kind__
    four cards with the same rank

__straight flush__
    five cards in sequence (as defined above) and with the same suit

The goal of this exercise is to estimate the probability of drawing these various hands.

1. Add methods named `haspair`, `hastwopair`, etc. that return true or false according to whether or not the hand meets the relevant criteria. Your code should work correctly for “hands” that contain any number of cards (although 5 and 7 are the most common sizes).

1. Write a method named `classify` that figures out the highest-value classification for a hand and sets the label field accordingly. For example, a 7-card hand might contain a flush and a pair; it should be labeled “flush”.

1. When you are convinced that your classification methods are working, the next step is to estimate the probabilities of the various hands. Write a function that shuffles a deck of cards, divides it into hands, classifies the hands, and counts the number of times various classifications appear.

1. Print a table of the classifications and their probabilities. Run your program with larger and larger numbers of hands until the output values converge to a reasonable degree of accuracy. Compare your results to the values at https://en.wikipedia.org/wiki/Hand_rankings.

*I didn't follow the instructions to the letter, but ultimately I came up with something that should deliver the same result.*

*First, some code from the book on which we'll build our functions:*

In [1]:
struct Card
    suit :: Int64
    rank :: Int64
    function Card(suit::Int64, rank::Int64)
        @assert(1 ≤ suit ≤ 4, 
            "suit is not between 1 and 4")
        @assert(1 ≤ rank ≤ 13,
            "rank is not between 1 and 13")
        new(suit, rank)
    end
end

In [2]:
const suit_names = ["♣", "♦", "♥", "♠"];
const rank_names = ["A", "2", "3", "4", "5", "6", "7", 
                    "8", "9", "10", "J", "Q", "K"];

In [3]:
function Base.show(io::IO, card::Card)
    print(io, rank_names[card.rank], suit_names[card.suit])
end

In [4]:
import Base.<

function <(c1::Card, c2::Card)
    (c1.suit, c1.rank) < (c2.suit, c2.rank)
end

< (generic function with 75 methods)

In [5]:
abstract type CardSet end

In [6]:
struct Deck <: CardSet
    cards :: Array{Card, 1}
end

function Deck()
    deck = Deck(Card[])
    for suit in 1:4
        for rank in 1:13
            push!(deck.cards, Card(suit, rank))
        end
    end
    deck
end

Deck

In [7]:
function Base.show(io::IO, deck::Deck)
    for card in deck.cards
        print(io, card, " ")
    end
    println()
end

In [8]:
function Base.pop!(deck::Deck)
    pop!(deck.cards)
end

In [9]:
function Base.push!(deck::Deck, card::Card)
    push!(deck.cards, card)
    deck
end

In [10]:
using Random

function Random.shuffle!(deck::Deck)
    shuffle!(deck.cards)
    deck
end

In [11]:
import Base.isless

function isless(c1::Card, c2::Card)
    isless((c1.suit, c1.rank), (c2.suit, c2.rank))
end

isless (generic function with 75 methods)

In [12]:
import Base.sort!

function sort!(deck::Deck)
    sort!(deck.cards)
    deck
end


sort! (generic function with 13 methods)

In [13]:
struct Hand <: CardSet
    cards :: Array{Card, 1}
    label :: String
end

function Hand(label::String = "")
    Hand(Card[], label)
end

Hand

In [14]:
hand = Hand("new hand")

Hand(Card[], "new hand")

In [15]:
function Base.show(io::IO, cs::CardSet)
    for card in cs.cards
        print(io, card, " ")
    end
end

function Base.pop!(cs::CardSet)
    pop!(cs.cards)
end

function Base.push!(cs::CardSet, card::Card)
    push!(cs.cards, card)
    nothing
end

In [16]:
function move!(cs1::CardSet, cs2::CardSet, n::Int)
    @assert 1 ≤ n ≤ length(cs1.cards)
    for i in 1:n
        card = pop!(cs1)
        push!(cs2, card)
    end
    nothing
end

move! (generic function with 1 method)

In [17]:
function Base.sort!(hand::Hand)
    sort!(hand.cards)
end

*My solution to __ex. 18.5__:*

In [18]:
"""
    deal!(deck :: Deck, numberhands :: Int64, numbercards :: Int64)

Shuffles and divides a deck into a specified number of hands, 
each with a specified number of cards.
"""

function deal!(deck :: Deck, numberhands :: Int64, numbercards :: Int64)
    
    @assert((numberhands * numbercards) ≤ length(deck.cards), 
        "not enough cards for this many hands")
    
    shuffled = Random.shuffle!(deck)
    
    allhands = []
    for i in 1:numberhands
        currentdeck = []
        for j in 1:numbercards
            
            push!(currentdeck, pop!(shuffled))
            
        end
        push!(allhands, currentdeck)
    end
    allhands
end

deal! (generic function with 1 method)

In [19]:
newdeck = Deck()

deal!(newdeck, 10, 5)

10-element Array{Any,1}:
 Any[9♦, 3♥, 9♣, 6♠, 9♥]  
 Any[7♣, 8♦, 6♥, 3♠, 4♠]  
 Any[K♠, 7♦, 7♠, K♣, 6♣]  
 Any[K♥, 3♦, 3♣, J♠, Q♠]  
 Any[2♥, J♦, A♦, J♥, K♦]  
 Any[9♠, 10♣, J♣, 6♦, 10♥]
 Any[Q♦, 5♣, 7♥, 2♦, A♥]  
 Any[5♥, 2♠, 4♣, Q♥, 8♣]  
 Any[A♠, 5♠, 4♥, Q♣, 5♦]  
 Any[8♥, 4♦, 8♠, A♣, 10♠] 

In [20]:
"""
    findmultiples(hand)

Tabulates the occurrence of cards with the same rank.
"""

function findmultiples(hand)
    d, d2 = Dict(), Dict(1 => 0, 2 => 0, 3 => 0, 4 => 0)

    for h in hand
        d[h.rank] = 1 + get(d, h.rank, 0)
    end

    for n in values(d)
        d2[n] = 1 + get(d2, n, 0)
    end

    d2
end

findmultiples (generic function with 1 method)

In [21]:
"""
    getranks(hand)

Creates an array with the ranks of the cards in a hand.

See also: [`findstreak`](@ref)
"""

function getranks(hand)
    ranks = []
    for h in hand
        push!(ranks, h.rank)
    end
    ranks = sort(ranks)
end

"""
    findstreak(hand)

Finds runs of cards with increasing ranks and returns the
maximum run.

See also: [`getranks`](@ref)
"""

function findstreak(hand)
    ranks = getranks(hand)
    streak = 1
    maxstreak = 1
    for i in 1:(length(ranks) - 1)
        if (ranks[i] + 1) == ranks[i + 1] 
            streak += 1
        elseif (ranks[i]) == ranks[i + 1]
            streak += 0
        else
            maxstreak = maximum((streak, maxstreak))
            streak = 1
        end
    end
    maxstreak = maximum((streak, maxstreak))
    maxstreak
end

findstreak (generic function with 1 method)

In [22]:
"""
    findstraight(hand)

Finds all varieties of straights in a hand.
"""

function findstraight(hand)
    streak = findstreak(hand)
    
    ranks = getranks(hand)
    
    if streak >= 5
        return true
    # find straights of 10, J, Q, K, A
    elseif (1, 10, 11, 12, 13) ⊆ ranks
        return true
    else
        return false
    end
end

findstraight (generic function with 1 method)

In [23]:
"""
    findflush(hand)

Finds flushes in a hand.
"""

function findflush(hand)
 
    d = Dict(1 => 0, 2 => 0, 3 => 0, 4 => 0)
    
    for h in hand
        d[h.suit] = 1 + get(d, h.suit, 0)
    end
    
    if maximum(values(d)) >= 5
        return true
    end
    false
end

findflush (generic function with 1 method)

In [24]:
"""
    findstraightflush(hand)

Finds straight flushs in a hand.
"""

# N.B. In 5-card poker, a hand that is 5-card straight and
# a 5-card flush would automatically be a straight flush.
# However, this is not necessarily the case in 7-card poker.
# This function will find straightflushes in both 5- and 7-card
# games.

function findstraightflush(hand)
    
    hand = sort(hand)
    straightflushstreak = 1
    maxstreak = 1
    for i in 1:(length(hand) - 1)
        if hand[i].suit == hand[i + 1].suit &&
            hand[i].rank + 1 == hand[i + 1].rank
            straightflushstreak += 1
        else
            maxstreak = maximum((straightflushstreak, 
                                 maxstreak))
            straightflushstreak = 1
        end
    end
    maxstreak = maximum((straightflushstreak, maxstreak))
    
    if maxstreak >= 5
        return true
    end
    false
end


findstraightflush (generic function with 1 method)

In [25]:
"""
    findroyalflush(hand)

Finds royalflush (run in one suit of 10, J, Q, K, A) in a hand.
"""

function findroyalflush(hand)
    hand = sort(hand)

    samesuit = [hand[1]]
    for h in 2:length(hand)
        if hand[h].suit == hand[h - 1].suit
            push!(samesuit, hand[h])
        else
            if length(samesuit) < 5
                samesuit = [hand[h]]
            end
        end
    end
    
    if length(samesuit) >= 5
        samesuitranks = []

        for s in samesuit
            push!(samesuitranks, s.rank)
        end

        if (1, 10, 11, 12, 13) ⊆ samesuitranks
            return true
        end
    end
    false
end

findroyalflush (generic function with 1 method)

In [26]:
"""
    classify(hand)

Tabulates the occurrences of various hands in poker.
"rf" = Royal Flush; "sf" = Straight Flush; "fk" = "Four of a Kind";
"fh" = Full House; "fl" = Flush; "st" = Straight; 
"tk" = Three of a Kind; "tp" = Two Pairs; "pr" = Pair; "np" = No Pair.
"""

probsofhands = Dict("rf" => 0, "sf" => 0, "fk" => 0, "fh" => 0, "fl" => 0,
                    "st" => 0, "tk" => 0, "tp" => 0, "pr" => 0, "np" => 0);



function classify(hand)
    
    mults = findmultiples(hand)
    
    if findroyalflush(hand)
        probsofhands["rf"] = 1 + get(probsofhands, "rf", 0)
    elseif findstraightflush(hand)
        probsofhands["sf"] = 1 + get(probsofhands, "sf", 0)
    elseif mults[4] == 1
        probsofhands["fk"] = 1 + get(probsofhands, "fk", 0)
    elseif mults[3] == 1 && mults[2] == 1
        probsofhands["fh"] = 1 + get(probsofhands, "fh", 0)
    elseif findflush(hand)
        probsofhands["fl"] = 1 + get(probsofhands, "fl", 0)
    elseif findstraight(hand)
        probsofhands["st"] = 1 + get(probsofhands, "st", 0)
    elseif mults[3] == 1 
        probsofhands["tk"] = 1 + get(probsofhands, "tk", 0)
    elseif mults[2] == 2
        probsofhands["tp"] = 1 + get(probsofhands, "tp", 0)
    elseif mults[2] == 1
        probsofhands["pr"] = 1 + get(probsofhands, "pr", 0)
    else
        probsofhands["np"] = 1 + get(probsofhands, "np", 0)
    end
probsofhands
end

classify (generic function with 1 method)

In [27]:
probsofhands = Dict("rf" => 0, "sf" => 0, "fk" => 0, "fh" => 0, "fl" => 0,
                    "st" => 0, "tk" => 0, "tp" => 0, "pr" => 0, "np" => 0);

# We may wish to break larger experiments (i.e., > 10,000,000 hands)
# into stages.  x will indicate which stage we're on

x = 0

0

In [28]:

handsperdeck = 10
totalhands = 3000000
cardsperhand = 5

for i in 1:(totalhands/handsperdeck)
    newdeck = Deck()

    mm = deal!(newdeck, handsperdeck, cardsperhand)


    for n in mm
        classify(n)
    end
end

x += 1
println("After ", x * totalhands, " total hands:")

probsofhands

After 3000000 total hands:


Dict{String,Int64} with 10 entries:
  "st" => 11829
  "sf" => 35
  "fh" => 4382
  "fl" => 5887
  "fk" => 709
  "tp" => 142606
  "np" => 1502791
  "pr" => 1268675
  "rf" => 3
  "tk" => 63083

In [29]:
for (t, s) in sort(collect(zip(values(probsofhands), keys(probsofhands))))
    println(s, " ", round((t/sum(values(probsofhands))) * 100, digits = 5), "%")
end

rf 0.0001%
sf 0.00117%
fk 0.02363%
fh 0.14607%
fl 0.19623%
st 0.3943%
tk 2.10277%
tp 4.75353%
pr 42.28917%
np 50.09303%


*Ran experiments on 3,000,000 5-card hands (took around a minute).  The numbers are very close to the theoretical percentages.*


rf 0.00017%

sf 0.0014%

fk 0.02313%

fh 0.14777%

fl 0.19257%

st 0.38953%

tk 2.11503%

tp 4.73043%

pr 42.24567%

np 50.1543%

*Ran experiments on 130,000,000 7-card hands (took considerably longer than a minute).  The numbers are very close to the theoretical percentages.*

rf 0.00475%

sf 0.02774%

fk 0.16745%

fh 2.46055%

fl 3.02523%

st 3.85077%

tk 4.9198%

np 19.85817%

tp 21.64887%

pr 44.03668%
